## Problem Set 6. Optimization methods. November 2015

### Makarova Anastasia
### Imports

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from numpy.matlib import repmat
import cvxpy as cvx
import gurobipy as grb

## Problem 2

Ticket-to-Ride (TicketToRide.m). You are opening a railroad company using
the existing railroad tracks. You can start operations along any of the section of the rail on the
map below. Each section you choose to operate will give you a certain yearly profit (always
positive, specified in TicketToRide.m). However, in any city that is adjacent to the sections you
operate, you need to open and maintain a depot, which has a yearly cost of 10 units. Using the
reduction to maxflow and an LP solver for the maxflow, find out the optimal strategy in this
situation (which sections to open). In your report put the map with the cities and/or sections
you decide to operate highlighted (use any graphical editor) for that.

In [54]:
import numpy as np
cities = ['Atlanta','Boston', 'Calgary', 'Charleston','Chicago','Dallas', 
          'Denver', 'Duluth', 'ElPaso','Helena', 'Houston', 'KansasCity', 'LasVegas',
          'LittleRock', 'LosAngeles', 'Miami', 'Montreal', 'Nashville',
          'NewOrleans', 'NewYork', 'OklahomaCity', 'Omaha', 'Phoenix',
          'Pittsburgh', 'Portland', 'Raleigh', 'SaintLouis','SaltLakeCity',
          'SanFrancisco','SantaFe', 'SaultStMarie', 'Seattle', 'Toronto',
          'Vancouver', 'Washington', 'Winnipeg'];

blue = ['Winnipeg','Helena',4,
    'Portland','SaltLakeCity',6,
    'Omaha','Chicago',4,
    'Montreal','NewYork',3,
    'KansasCity','SaintLouis',2,
    'SantaFe','OklahomaCity',3,
    'Atlanta','Miami',5];

blue = GetMatrix(blue,cities);

white = ['Calgary','Winnipeg',6,
    'Chicago','Toronto',4,
    'SaintLouis','Chicago',2,
    'Pittsburgh','NewYork',2,
    'SanFrancisco','SaltLakeCity',5,
    'Phoenix','Denver',5,
    'LittleRock','Nashville',3];

white = GetMatrix(white,cities);

green = ['Portland','SanFrancisco',5,
    'Helena','Denver',4,
    'SaintLouis','Pittsburgh',5,
    'LittleRock','NewOrleans',3,
    'ElPaso','Houston',6];

green = GetMatrix(green,cities);

red = ['Helena','Omaha',5,
    'Denver','OklahomaCity',4,
    'ElPaso','Dallas',4,
    'NewOrleans','Miami',6,
    'NewYork','Boston',2,
    'Duluth','Chicago',3];

red = GetMatrix(red,cities);

purple = ['Helena','SaltLakeCity',3,
    'Denver','Omaha',4,
    'Duluth','Toronto',6,
    'Charleston','Miami',4,
    'SanFrancisco','LosAngeles',3];
purple = GetMatrix(purple,cities);

orange = ['LasVegas','SaltLakeCity',3,
    'Helena','Duluth',6,
    'Chicago','Pittsburgh',3,
    'Denver','KansasCity',4,
    'NewYork','Washington',2,
    'Atlanta','NewOrleans',4];

orange = GetMatrix(orange,cities);

black = ['LosAngeles','ElPaso',6,
    'Winnipeg','Duluth',4,
    'SaultStMarie','Montreal',5,
    'Nashville','Raleigh',3];

black = GetMatrix(black,cities);

yellow = ['Seattle','Helena',6,
    'ElPaso','OklahomaCity',5,
    'Nashville','Pittsburgh',4];

yellow = GetMatrix(yellow,cities);

gray = ['Vancouver','Seattle',1,
    'Portland','Seattle',1,
    'Vancouver','Calgary',3,
    'Seattle','Calgary',4,
    'Calgary','Helena',4,
    'Winnipeg','SaultStMarie',6,
    'Duluth','SaultStMarie',3,
    'SaultStMarie','Toronto',2,
    'Toronto','Montreal',3,
    'Montreal','Boston',2,
    'Toronto','Pittsburgh',2,
    'LasVegas','LosAngeles',2,
    'LosAngeles','Phoenix',3,
    'Phoenix','ElPaso',3,
    'ElPaso','SantaFe',2,
    'Phoenix','SantaFe',3,
    'SantaFe','Denver',2,
    'Duluth','Omaha',2,
    'Omaha','KansasCity',1,
    'KansasCity','OklahomaCity',2,
    'OklahomaCity','Dallas',2,
    'Dallas','Houston',1,
    'Houston','NewOrleans',2,
    'Dallas','LittleRock',2,
    'OklahomaCity','LittleRock',2,
    'LittleRock','SaintLouis',2,
    'SaintLouis','Nashville',2,
    'Nashville','Atlanta',1,
    'Atlanta','Charleston',2,
    'Charleston','Raleigh',2,
    'Atlanta','Raleigh',2,
    'Raleigh','Washington',2,
    'Raleigh','Pittsburgh',2,
    'Washington','Pittsburgh',2];

gray = GetMatrix(gray,cities);

profit = blue*1.0+white*1.5+green*2.0+red*2.0+purple*2.5+orange*1.0+black*1.0+yellow*2.0+gray*1.0;


#TODO: symmetrize the matrix if necessary by uncommenting the next line
#profit_sym = np.dstack([profit, profit.transpose()]).max(axis=2)

costs = np.ones((len(cities),1))*10;

#TODO put your solution here

In [2]:
def GetMatrix(links,cities):

    A = np.zeros((len(cities), len(cities)));
    for i in xrange(0, len(links), 3):
        A[cities.index(links[i]), cities.index(links[i + 1])] =  links[i+2]
           
    return A        

* The initial problem 

\begin{equation*}
\begin{aligned}
& \underset{x \ in \{0,1\}}{\text{maximize}}
& & \sum_{i,j } u_{ij}x_ix_j - 10 \sum_ix_i
\end{aligned}
\end{equation*}

where $u_{ij}$ - elements of symmetrix matrix of segments profit

* Maxflow problem

\begin{equation*}
\begin{aligned}
& \underset{x \ in \{0,1\}}{\text{maximize}}
& & \sum_{i,j } u_{ij}x_i - 10 \sum_ix_i - \sum_{i,j } u_{ij}max\{x_j - x_i, 0\}
\end{aligned}
\end{equation*}

where the last sum can be considered penalty for closed city when the road segment is used.

* Binary minimization with pairewise terms

\begin{equation*}
\begin{aligned}
& \underset{x \ in \{0,1\}}{\text{minimize}}
& & \sum_{i} u_{i}x_i + \sum_{i,j } u_{ij}max\{x_j - x_i, 0\} \\
& & & u_i = 10 - \sum_ju_{ji}
\end{aligned}
\end{equation*}

Also we have to add 2 additional cities - source and sink with $ x_s = 0$ and $x_t = 1$. Then the edges from source to vertice $i$ will cost $max(0, u_i)$, the edges from vertice $i$ to sink will cost $max(0, -u_i)$ and will be included into the minimization problem above.


In [64]:
nmb_c = len(cities)

open_cost = 10

city_profit = open_cost - sum(profit[:, 0:nmb_c ])

U = np.zeros((nmb_c + 2, nmb_c + 2))
U[1:(nmb_c + 1), 1: (nmb_c + 1)] = profit


pos = np.arange(nmb_c)[city_profit > 0]
neg = np.arange(nmb_c)[city_profit < 0]

# for source and sink
for i in pos:
    U[0, i + 1] = city_profit[i]
for i in neg:
    U[i + 1, nmb_c + 1] = - city_profit[i]


In [76]:
X = cvx.Variable(nmb_c + 2)

f = 0
for i, j in np.ndindex(nmb_c + 2, nmb_c + 2):
    f += U[i, j] * cvx.pos(X[j] - X[i])

objective = cvx.Minimize(f)

constraints = [X >= 0, X <= 1, X[0] == 0, X[-1] == 1]

solution = cvx.Problem(objective, constraints)
solution.solve(solver='GUROBI')

opened_cities = list(np.array(cities)[np.array(X.value).astype(bool)[1:-1].ravel()])
print('Optimal value is: {0}'.format(solution.value))
print("Opened {0} cities: {1}".format(len(opened_cities), opened_cities))

index = [cities.index(city) for city in opened_cities]
print("\n Optimal value of year profit is: {0}".format(np.sum([profit[i, j] 
                                            for i in ids for j in index]) - opening_cost * len(opened_depots))) 


Optimal value is: 86.0
Opened 27 cities: ['Atlanta', 'Calgary', 'Charleston', 'Chicago', 'Dallas', 'Denver', 'Duluth', 'ElPaso', 'Helena', 'Houston', 'LittleRock', 'LosAngeles', 'Miami', 'Nashville', 'NewOrleans', 'OklahomaCity', 'Omaha', 'Phoenix', 'Pittsburgh', 'Portland', 'SaintLouis', 'SaltLakeCity', 'SanFrancisco', 'SaultStMarie', 'Seattle', 'Toronto', 'Winnipeg']

 Optimal value of year profit is: 25.5
